In [173]:
 #경로 설정
import os
os.chdir('/content/drive/My Drive/dacon/20201204_NLP')

In [174]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [175]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [176]:
from matplotlib import rcParams, pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import re
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, GlobalMaxPooling1D, Conv1D, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam
import warnings 
warnings.filterwarnings(action='ignore')
#추가
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
import seaborn as sns
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [177]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)
else:
    print('No GPU detected')

1 Physical GPUs, 1 Logical GPU


In [178]:
rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

In [179]:
data_dir = Path('/content/drive/My Drive/dacon/20201204_NLP')
feature_dir = Path('../build/feature')
val_dir = Path('../build/val')
tst_dir = Path('../build/tst')
sub_dir = Path('../build/sub')

dirs = [feature_dir, val_dir, tst_dir, sub_dir]
for d in dirs:
    os.makedirs(d, exist_ok=True)

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test_x.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'author'
n_fold = 5
n_class = 5
seed = 42

In [180]:
algo_name = 'lstm_1129'
feature_name = 'emb'
model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv'
p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'
sub_file = sub_dir / f'{model_name}.csv'

In [181]:
trn = pd.read_csv(trn_file, index_col=0)
tst = pd.read_csv(tst_file, index_col=0)

In [182]:
def data_text_cleaning(text):
 
    # 영문자 이외 문자는 공백으로 변환
    only_english = re.sub('[^A-Za-z0-9 ]', ' ', text)
 
    # 소문자 변환
    no_capitals = only_english.lower().split()
 
    # 불용어 제거
    stops = set(stopwords.words('english'))
    no_stops = [word for word in no_capitals if not word in stops]
 
    # 어간 추출
    stemmer = nltk.stem.SnowballStemmer('english')
    stemmer_words = [stemmer.stem(word) for word in no_stops]
 
    # 공백으로 구분된 문자열로 결합하여 결과 반환
    return ' '.join(stemmer_words)

In [183]:
trn['text']=trn['text'].apply(data_text_cleaning)
tst['text']=tst['text'].apply(data_text_cleaning)

In [184]:
clear_punc = []
for t in range(len(trn.text)):
  clear_punc.append(re.sub('[!@#$%^&“*()_+”=‘,./ª?\><;":`~]', ' ' ,trn.text[t]))
trn['clear_punc'] = clear_punc

In [185]:
clear_punc = []
for t in range(len(tst.text)):
  clear_punc.append(re.sub('[!@#$%“^&*()_+=,‘./ª?\><;":`~]', ' ' ,tst.text[t]))
tst['clear_punc'] = clear_punc

In [187]:
trn['length_text'] = trn.clear_punc.apply(lambda x : len(x))
tst['length_text'] = tst.clear_punc.apply(lambda x : len(x))

In [188]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer('\s+', gaps = True)
token_text = []

for t in trn.clear_punc:
  temp = tokenizer.tokenize(t)
  #temp = [word for word in temp if not word in stopwords]
  token_text.append(temp)

trn['token_text'] = token_text

In [189]:
token_text = []

for t in tst.clear_punc:
  temp = tokenizer.tokenize(t)
  #temp = [word for word in temp if not word in stopwords]
  token_text.append(temp)

tst['token_text'] = token_text

In [190]:
trn['length_token'] = trn.token_text.apply(lambda x : len(x))
tst['length_token'] = tst.token_text.apply(lambda x : len(x))

In [191]:
trn = trn.loc[trn.length_token > 6]

In [139]:
trn.index = range(trn.shape[0])

In [192]:
trn

,text,author,clear_punc,length_text,token_text,length_token
index,,,,,,
0,almost choke much much want say strang exclam ...,3,almost choke much much want say strang exclam ...,107,"[almost, choke, much, much, want, say, strang,...",20
2,engag one day walk perus jane last letter dwel...,1,engag one day walk perus jane last letter dwel...,162,"[engag, one, day, walk, perus, jane, last, let...",29
3,captain porch keep care way treacher shot inte...,4,captain porch keep care way treacher shot inte...,169,"[captain, porch, keep, care, way, treacher, sh...",31
4,merci gentlemen odin flung hand write anyway s...,3,merci gentlemen odin flung hand write anyway s...,106,"[merci, gentlemen, odin, flung, hand, write, a...",18
5,well fought said sooth charg us twice,4,well fought said sooth charg us twice,37,"[well, fought, said, sooth, charg, us, twice]",7
...,...,...,...,...,...,...
54871,sir odin great caus thank said princ,4,sir odin great caus thank said princ,36,"[sir, odin, great, caus, thank, said, princ]",7
54873,look back shoulder beckon enter mr odin got ar...,0,look back shoulder beckon enter mr odin got ar...,89,"[look, back, shoulder, beckon, enter, mr, odin...",16
54874,mr smith odin whisper hard dare hope would come,2,mr smith odin whisper hard dare hope would come,47,"[mr, smith, odin, whisper, hard, dare, hope, w...",9


In [193]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [194]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(trn.token_text)

In [195]:
threshold = 6
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 19576
등장 빈도가 5번 이하인 희귀 단어의 수: 11073
단어 집합에서 희귀 단어의 비율: 56.56416019615856
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 2.20160674252675


In [196]:
vocab_size = total_cnt - rare_cnt + 1 # 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거. 0번 패딩 토큰을 고려하여 +1
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 8504


In [197]:
tokenizer = Tokenizer(vocab_size,oov_token = 'oov')
tokenizer.fit_on_texts(trn.token_text)

In [198]:
def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

In [199]:
below_threshold_len(300, trn.token_text)

전체 샘플 중 길이가 300 이하인 샘플의 비율: 100.0


In [200]:
y = np.array([x for x in trn['author']])

In [201]:
max_len =  300

In [202]:
trn = tokenizer.texts_to_sequences(trn.token_text)
trn = pad_sequences(seq, maxlen = max_len, padding = 'post')

In [203]:
tst = tokenizer.texts_to_sequences(tst.token_text)
tst = pad_sequences(target, maxlen = max_len, padding = 'post')

In [204]:
# load the whole embedding into memory
glove = dict()
f = open('/content/drive/MyDrive/dacon/20201204_NLP/glove.txt')
for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:], dtype='float32')
    glove[word] = vector
f.close()

In [205]:
embedding_matrix = np.zeros((vocab_size, 100)) #100차원의 임베딩 매트릭스 생성

for index, word in enumerate(word): #vocabulary에 있는 토큰들을 하나씩 넘겨줍니다.
    if word in glove: #넘겨 받은 토큰이 word2vec에 존재하면(이미 훈련이 된 토큰이라는 뜻)
        embedding_vector = glove[word] #해당 토큰에 해당하는 vector를 불러오고
        embedding_matrix[i] = embedding_vector #해당 위치의 embedding_mxtrix에 저장합니다.
    else:
        print("glove 없는 단어입니다.")
        break

**lstm 모델**

In [211]:
embedding_dim = 64

In [212]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [213]:
def get_model():
    model = Sequential([
        Embedding(vocab_size, embedding_dim, weights = [embedding_matrix], input_length=max_len),
        Dropout(.2),
        Bidirectional(LSTM(64, return_sequences=True)),
        Bidirectional(LSTM(64)),
        Dense(n_class, activation='softmax')
    ])
    
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=.01))
    return model

In [214]:
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
    print(f'training model for CV #{i}')
    clf = get_model()
    
    es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)

    clf.fit(trn[i_trn], 
            to_categorical(y[i_trn]),
            validation_data=(trn[i_val], to_categorical(y[i_val])),
            epochs=10,
            batch_size=512,
            callbacks=[es])
    p_val[i_val, :] = clf.predict(trn[i_val])
    p_tst += clf.predict(tst) / n_fold

ValueError: ignored

In [ ]:
print(f'Accuracy (CV): {accuracy_score(y, np.argmax(p_val, axis=1)) * 100:8.4f}%')
print(f'Log Loss (CV): {log_loss(pd.get_dummies(y), p_val):8.4f}')

In [ ]:
np.savetxt(p_val_file, p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, p_tst, fmt='%.6f', delimiter=',')

In [ ]:
print(clf.summary())

In [ ]:
plot_model(clf)

In [ ]:
sub = pd.read_csv(sample_file, index_col=0)
print(sub.shape)
sub.head()

In [ ]:
sub[sub.columns] = p_tst
sub.head()

In [ ]:
sub.to_csv(sub_file)